# Generating Speech

## "Get me a flight to JFK"

In [ ]:
from gtts import gTTS
import os

mytext = 'Good Morning. I need to fly to New York and say over in London.\
           Get me a flight to JFK and a hotel in central London'

language = 'en'

tts = gTTS(text=mytext, lang=language) 

tts.save('flight.mp3')

os.system("mpg321 flight.mp3")

In [ ]:
# convert to wav file 
from pydub import AudioSegment
from pydub.playback import play

# files                                                                         
src = "flight.mp3"
dst = "flight.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)

sound.export(dst, format="wav")
play(sound)


# Transcribing Speech

In [ ]:
import speech_recognition as sr

r = sr.Recognizer()

with sr.WavFile('flight.wav') as source:
     audio = r.record(source)  # read the entire audio file 
     print("Transcription: " + r.recognize_google(audio, language='en-US'))

# Understanding Grammar

In [ ]:
import nltk
from nltk import pos_tag, word_tokenize

sentence = "Get me a flight to JFK"

tokens = word_tokenize(sentence)
tagged_tokens = pos_tag(tokens)
print(tagged_tokens)


# What can go wrong?

## Syntactic ambiguity

In [ ]:
from nltk import ne_chunk, pos_tag, word_tokenize
utterance = ("Get me a flight to JFK")
tree = ne_chunk(pos_tag(word_tokenize(utterance)))
print(tree)

### Print dependency parsing tree

In [ ]:
import nltk
from nltk import Tree
from nltk.draw.util import CanvasFrame
from nltk.draw import TreeWidget
from IPython.display import display
from IPython.display import Image

grammar = "NP: {<DT>?<JJ>*<NN>}"
parser = nltk.RegexpParser(grammar)
result = parser.parse(tree)

cf = CanvasFrame()
tc = TreeWidget(cf.canvas(),result)
cf.add_widget(tc,10,10) # (10,10) offsets
cf.print_to_file('tree.ps')
cf.destroy()

! convert tree.ps tree.png
! rm tree.ps
image = Image(filename='tree.png')
display(image)
! rm tree.png

## Semantic ambiguity

In [ ]:
utterance = ("Get me a flight to John F Kennedy")
print(ne_chunk(pos_tag(word_tokenize(utterance))))

# Applying knowledge

In [ ]:
import json
import urllib

query = "JFK"
api_key = open('.api_key').read()
service_url = 'https://kgsearch.googleapis.com/v1/entities:search'
params = {
    'query': query,
    'limit': 10,
    'indent': True,
    'key': api_key,
}
url = service_url + '?' + urllib.parse.urlencode(params)
response = json.loads(urllib.request.urlopen(url).read())

for element in response['itemListElement']: 
    if 'result' in element and 'name' in element['result']:
        print(element['result']['name'] + ' (' + str(element['resultScore']) + ')')
